In [3]:
import os

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir('/content/drive/MyDrive/falando_nela_v2/src')
    print("Current working directory (Colab):", os.getcwd())
else:
  print("Current working directory (not Colab):", os.getcwd())

DB_PATH = "../data/DiscursosSenadores.sqlite"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Current working directory (Colab): /content/drive/MyDrive/falando_nela_v2/src


In [ ]:
# prompt: Ver schema do banco de dados
import sqlite3

def show_db_schema(db_path):
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()

        for table_name in tables:
            table_name = table_name[0]
            print(f"\nTable: {table_name}")
            cursor.execute(f"PRAGMA table_info({table_name});")
            columns = cursor.fetchall()
            for col in columns:
                print(f"  Column: {col[1]} ({col[2]})")  # name and type

        conn.close()

    except sqlite3.Error as e:
        print(f"Error accessing database: {e}")

show_db_schema(DB_PATH)



Table: Senadores
  Column: CodigoParlamentar (INTEGER)
  Column: NomeParlamentar (TEXT)
  Column: NomeCompletoParlamentar (TEXT)
  Column: SexoParlamentar (TEXT)
  Column: SiglaPartidoParlamentar (TEXT)
  Column: UfParlamentar (TEXT)
  Column: UrlFotoParlamentar (TEXT)
  Column: EmailParlamentar (TEXT)
  Column: NomeProfissao (TEXT)
  Column: IndicadorAtividadePrincipal (TEXT)

Table: sqlite_sequence
  Column: name ()
  Column: seq ()

Table: SenadoresCargos
  Column: IdCargo (INTEGER)
  Column: CodigoParlamentar (INTEGER)
  Column: NomeCargo (TEXT)
  Column: DataInicio (TEXT)
  Column: DataFim (TEXT)
  Column: Orgao (TEXT)

Table: SenadoresHistoricoAcademico
  Column: IdHistoricoAcademico (INTEGER)
  Column: CodigoParlamentar (INTEGER)
  Column: NomeCurso (TEXT)
  Column: NivelCurso (TEXT)
  Column: InstituicaoEnsino (TEXT)
  Column: AnoConclusao (TEXT)

Table: DiscursosAparteantes
  Column: Id (INTEGER)
  Column: CodigoPronunciamento (INTEGER)
  Column: CodigoParlamentar (INTEGER)
 

In [ ]:
# prompt: Ver head de DiscursosAnalises, coluna TopicosConstituicao

import sqlite3
import pandas as pd

DB_PATH = "../data/DiscursosSenadores.sqlite"

def ver_head_topicos(db_path, n=5):
    try:
        conn = sqlite3.connect(db_path)
        query = "SELECT TopicosConstituicao FROM DiscursosAnalises LIMIT ?"
        df = pd.read_sql_query(query, conn, params=[n])
        conn.close()
        return df
    except sqlite3.Error as e:
        print(f"Error accessing database: {e}")
        return None

df_topicos = print(ver_head_topicos(DB_PATH))



  TopicosConstituicao
0                None
1                  19
2                None
3                   3
4                None


In [4]:
!pip install xlsxwriter
# prompt: Quero exportar para xlsx as seguintes tabelas e colunas:
# Senadores:
#   Column: CodigoParlamentar (INTEGER)
#   Column: NomeParlamentar (TEXT)
#   Column: UfParlamentar (TEXT)
# Table: Discursos
#   Column: CodigoPronunciamento (INTEGER)
#   Column: CodigoParlamentar (INTEGER)
#   Column: DataPronunciamento (DATE)
#   Column: TextoResumo (TEXT)
#   Column: Indexacao (TEXT)
#   Column: SiglaPartidoParlamentarNaData (TEXT)
#   Column: UfParlamentarNaData (TEXT)
#   Column: SiglaCasaPronunciamento (TEXT)
#   Column: Forma (TEXT)
#   Column: TextoIntegral (TEXT)
# Table: DiscursosAnalises
#   Column: CodigoPronunciamento (INTEGER)
#   Column: BM25_Constituição (REAL)
#   Column: TFIDF_Constituição (REAL)
#   Column: SentimentoGeral (TEXT)
#   Column: SentimentoConstituicao (TEXT)
#   Column: SumarioConstituicao (TEXT)
#   Column: TrechosConstituicao (TEXT)
#   Column: NovaConstituinteOuConstituicao_resposta (TEXT)
#   Column: NovaConstituinteOuConstituicao_trecho (TEXT)
#   Column: TopicosConstituicao (TEXT)
# Table: DiscursosPesquisa
#   Column: CodigoPronunciamento (INTEGER)
#   Column: Similaridade_base_democracia (REAL)
#   Column: Similaridade_ultrapassada (REAL)
#   Column: Similaridade_prejudica_economia (REAL)
#   Column: Similaridade_direitos_demais (REAL)
#   Column: Similaridade_nova_constituinte (REAL)
#   Column: Similaridade_governo_nao_respeita (REAL)
#   Column: Similaridade_camara_nao_respeita (REAL)
#   Column: Similaridade_supremo_nao_respeita (REAL)
#   Column: Similaridade_ninguem_respeita (REAL)

import pandas as pd
import sqlite3

DB_PATH = "../data/DiscursosSenadores.sqlite"

def export_to_excel(db_path, output_file="falando_nela_1.xlsx"):
    try:
        conn = sqlite3.connect(db_path)

        # Senadores
        senadores_df = pd.read_sql_query("SELECT CodigoParlamentar, NomeParlamentar, UfParlamentar FROM Senadores", conn)

        # Discursos
        discursos_df = pd.read_sql_query("SELECT CodigoPronunciamento, CodigoParlamentar, DataPronunciamento, TextoResumo, Indexacao, SiglaPartidoParlamentarNaData, UfParlamentarNaData, SiglaCasaPronunciamento, Forma, TextoIntegral FROM Discursos", conn)

        # DiscursosAnalises
        discursos_analises_df = pd.read_sql_query("SELECT CodigoPronunciamento, BM25_Constituição, TFIDF_Constituição, SentimentoGeral, SentimentoConstituicao, SumarioConstituicao, TrechosConstituicao, NovaConstituinteOuConstituicao_resposta, NovaConstituinteOuConstituicao_trecho, TopicosConstituicao FROM DiscursosAnalises", conn)

        # DiscursosPesquisa
        discursos_pesquisa_df = pd.read_sql_query("SELECT CodigoPronunciamento, Similaridade_base_democracia, Similaridade_ultrapassada, Similaridade_prejudica_economia, Similaridade_direitos_demais, Similaridade_nova_constituinte, Similaridade_governo_nao_respeita, Similaridade_camara_nao_respeita, Similaridade_supremo_nao_respeita, Similaridade_ninguem_respeita FROM DiscursosPesquisa", conn)

        with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
            senadores_df.to_excel(writer, sheet_name="Senadores", index=False)
            discursos_df.to_excel(writer, sheet_name="Discursos", index=False)
            discursos_analises_df.to_excel(writer, sheet_name="DiscursosAnalises", index=False)
            discursos_pesquisa_df.to_excel(writer, sheet_name="DiscursosPesquisa", index=False)

        print(f"Data exported to {output_file}")
        conn.close()
    except sqlite3.Error as e:
        print(f"Error accessing database: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

export_to_excel(DB_PATH)


Data exported to banco_para_analise.xlsx
